In [1]:
#Get all our imports done with
import cv2
import numpy as np
import matplotlib as mplot
import os
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
#This whole block of code is sloppy and inefficient and has code that defeats the point of using other code, but it gets the job done for now. TODO: Format, cleanup unneeded bits of code

#Organizes the files into lists and a dict we can use for our algorithms and defines a function to 

#Change directory to the directory our data is stored in
os.chdir("E:\Users\Elliott\Documents\CKData\Emotion")
print("Current directory is: " + os.getcwd())

#Import emotion labels 
emotionIDList = ["Neutral", "Anger", "Contempt", "Disgust", "Fear", "Happiness", "Sadness", "Surprise"]
#Initialize dictionary and lists for storing emotion data and image file locations 
ImageIDEmotionPair = {}
peakEmotionFileLocations = []
neutralEmotionFileLocations = []

emotionsRootDir = "E:\Users\Elliott\Documents\CKData\Emotion"
imagesRootDir = "E:\Users\Elliott\Documents\CKData\extended-cohn-kanade-images\cohn-kanade-images"
for dirName, subDirList, fileList in os.walk(emotionsRootDir):
    if len(fileList) > 0:
        for fileName in fileList:
            #Change the directory to the directory containing the file we want to access
            lowestDir = os.path.normpath(fileName[:9].replace("_", "/"))
            os.chdir('E:\Users\Elliott\Documents\CKData\Emotion\%s' % (lowestDir))
            #Read the text from the .txt file that denotes the emotion of the penultimate image of a sequence
            fileEmotionTxt = open(fileName, "r")
            emotionNumID = int(fileEmotionTxt.read()[3:4])
            fileEmotionTxt.close()
            #Put the Photo IDs and the corresponding emotions(Number converted to a string of the emotion) together 
            #into a dict as a key:value pair
            slicedFileName = fileName[:17]
            ImageIDEmotionPair[slicedFileName] = emotionIDList[emotionNumID]
            neutralImage = imagesRootDir + os.path.normpath("/" + lowestDir + "/" + fileName[:15] + "00")
            #print(neutralImage)
            #Create a list of all the files with peak emotion and a list of all the neutral files (Could do this better in another os.walk for loop)
            peakEmotionFileLocations.append(imagesRootDir + os.path.normpath("/" + lowestDir + "/" + fileName[:17]))
            
#Iterating through the list directly with "for x in list" doesn't work here, hence the different for loop
for x in range(0, len(neutralEmotionFileLocations)):
    neutralEmotionFileLocations[x] = neutralEmotionFileLocations[x] + "00"
    
def getFileEmotion(fileLocation):
    fileLocation = fileLocation.replace(imagesRootDir, "")
    editedFileLocation = fileLocation[10:]
    return ImageIDEmotionPair[editedFileLocation]

print(getFileEmotion(peakEmotionFileLocations[0]))
print("\n\n\nPrinting dictionary of image file/emotion pairs:\n%s" % len(sorted(ImageIDEmotionPair.values())))
#print("\n" + getFileEmotion(peakEmotionFileLocations[0]))

    
    
    

Current directory is: E:\Users\Elliott\Documents\CKData\Emotion
Disgust



Printing dictionary of image file/emotion pairs:
327


In [3]:
#Convert the images to grayscale and isolate the faces --- Only needed to run once!!!

# #based off of http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/
# faceDet = cv2.CascadeClassifier("E:\Users\Elliott\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml")
# faceDet_two = cv2.CascadeClassifier("E:\Users\Elliott\opencv\sources\data\haarcascades\haarcascade_frontalface_alt2.xml")
# faceDet_three = cv2.CascadeClassifier("E:\Users\Elliott\opencv\sources\data\haarcascades\haarcascade_frontalface_alt.xml")
# faceDet_four = cv2.CascadeClassifier("E:\Users\Elliott\opencv\sources\data\haarcascades\haarcascade_frontalface_alt_tree.xml")


# def detectFaces(emotion):
#     filenumber = 0
#     currentEmotionFilesList = []
#     for fileName in peakEmotionFileLocations:
#         if getFileEmotion(fileName) == emotion:
#             currentEmotionFilesList.append(fileName)
#     for aFile in currentEmotionFilesList:
#         aFile = aFile + ".png"
#         #Image reading and conversion to grayscale 
#         frame = cv2.imread(aFile) #Open image
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
        
#         #Detect face using 4 different classifiers
#         face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
#         face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
#         face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
#         face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

        
#     #Go over detected faces, stop at first detected face, return empty if no face.
#         if len(face) == 1:
#             facefeatures = face
#         elif len(face_two) == 1:
#             facefeatures = face_two
#         elif len(face_three) == 1:
#             facefeatures = face_three
#         elif len(face_four) == 1:
#             facefeatures = face_four
#         else:
#             facefeatures = ""
    
#     #Cut and save face
#         for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
#             print "Face found in file: %s" % aFile
#             gray = gray[y:y+h, x:x+w] #Cut the frame to size
        
#             try:
#                 out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
#                 print("Writing file...")
#                 cv2.imwrite("E:\\Users\\Elliott\\Documents\\CKData\\EditedGrayScaleFaces\\%s\\%s.jpg" %(emotion, filenumber), out) #Write image
#             except:
#                 pass #If error, pass file
#         filenumber += 1 #Increment image number
        
# #Edited to avoid neutral until made compatible
# for emotion in emotionIDList[1:8]:
#     os.makedirs("E:\\Users\\Elliott\\Documents\\CKData\\EditedGrayScaleFaces\\%s" % emotion)
#     print(emotion)
#     detectFaces(emotion) #Call function
       

In [4]:
#Organize the data into training and prediction sets
#THIS PIECE OF CODE CAN PROBABLY ALSO BE IMPLEMENTED MORE EFFICIENTLY
from random import shuffle
def splitEmotionData(emotion):
    currentDir = "E:\\Users\\Elliott\\Documents\\CKData\\EditedGrayScaleFaces\\%s" % emotion
    currentEmotionFilesList = []
    for aFile in os.listdir(currentDir):
        currentEmotionFilesList.append(currentDir + os.path.normpath("/" + aFile))
    shuffle(currentEmotionFilesList)
    trainingSet = currentEmotionFilesList[:int(len(currentEmotionFilesList) * .8)]
    testingSet = currentEmotionFilesList[-int(len(currentEmotionFilesList) * .2):]
#     print(len(trainingSet))
#     print(len(testingSet))
    #Length is one off but we worry about that later
#     print(len(currentEmotionFilesList))
    
    return trainingSet, testingSet
    
def createSets():
    trainingSetFiles = []
    trainingEmotions = []
    testingSetFiles = []
    testingEmotions = []
    for emotion in emotionIDList[1:8]:
        training, testing = splitEmotionData(emotion)
        #Create emotion labels for each set
        for item in training:
            item = item
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            trainingSetFiles.append(gray) #append image array to training data list
            trainingEmotions.append(emotionIDList.index(emotion))
        for item in testing:
            item = item
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            testingSetFiles.append(gray)
            testingEmotions.append(emotionIDList.index(emotion))
            
    return trainingSetFiles, trainingEmotions, testingSetFiles, testingEmotions

In [10]:
#run tests
fisherFaces = cv2.createFisherFaceRecognizer()

#http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/
def runFisherFaces():
    trainingSetFiles, trainingEmotions, testingSetFiles, testingEmotions = createSets()
    incorrectEmotions = {}
    correctEmotions = {}
    for emotion in emotionIDList[1:8]:
        incorrectEmotions[emotion] = 0
        correctEmotions[emotion] = 0
    print "\n\n\nTraining fisher face classifier..."
    print "Size of training set is:", len(trainingEmotions), "images"
    fisherFaces.train(trainingSetFiles, np.asarray(trainingEmotions))

    print "Predicting classification set..."
    count = 0
    correct = 0
    incorrect = 0
    for image in testingSetFiles:
        pred, conf = fisherFaces.predict(image)
        if pred == testingEmotions[count]:
            correctEmotions[emotionIDList[testingEmotions[count]]] += 1
            correct += 1
            count += 1
        else:
            incorrectEmotions[emotionIDList[testingEmotions[count]]] += 1
            incorrect += 1
            count += 1
    return ((100*correct)/(correct + incorrect)), incorrectEmotions, correctEmotions

#Now run it
metascore = []
# confMatrix = {}
# for emotion in emotionIDList[1:7]:
#     tempList = []
#     confMatrix[emotion] = tempList
for i in range(0,1):
    correct, amountIncorrectPerEmotion, amountCorrectPerEmotion = runFisherFaces()
    print ("Got %s percent correct!" % str(correct))
    for x in range(1, len(emotionIDList[1:8]) + 1):
        amountCorrect = amountCorrectPerEmotion[emotionIDList[x]]
        amountIncorrect = amountIncorrectPerEmotion[emotionIDList[x]]
        
        percentCorrectForEmotion = (100 * amountCorrect) / (amountCorrect + amountIncorrect)
        percentIncorrectForEmotion = (100 - percentCorrectForEmotion)
        print percentIncorrectForEmotion
        print("Amount correct for emotion(%s): %d" % (emotionIDList[x], amountCorrect))
        
        print("Percent of this emotion(%s) gotten correct: %.2f" % (emotionIDList[x], percentCorrectForEmotion))
    
        print 100 - correct, "percent incorrect!"
        print("Amount incorrect for emotion(%s): %d" % (emotionIDList[x], amountIncorrectPerEmotion[emotionIDList[x]]))
        percentIncorrectForEmotion = ()
        print("Percent of this emotion(%s) gotten incorrect: %.2f" % (emotionIDList[x], percentIncorrectForEmotion))
    
    metascore.append(correct)

print "\nEnd score:", np.mean(metascore), "percent correct!"






Training fisher face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 66 percent correct!
45
Amount correct for emotion(Anger): 5
Percent of this emotion(Anger) gotten correct: 55.00
34 percent incorrect!
Amount incorrect for emotion(Anger): 4


TypeError: float argument required, not tuple

In [18]:
eigenFaces = cv2.createEigenFaceRecognizer()

def runEigenFaces():
    trainingSetFiles, trainingEmotions, testingSetFiles, testingEmotions = createSets()
    
    print "Training Eigen face classifier..."
    print "Size of training set is:", len(trainingEmotions), "images"
    eigenFaces.train(trainingSetFiles, np.asarray(trainingEmotions))

    print "Predicting classification set..."
    count = 0
    correct = 0
    incorrect = 0
    for image in testingSetFiles:
        pred, conf = eigenFaces.predict(image)
        if pred == testingEmotions[count]:
            correct += 1
            count += 1
        else:
            incorrect += 1
            count += 1
    return ((100*correct)/(correct + incorrect))

#Now run it
metascore = []
for i in range(0,10):
    correct = runEigenFaces()
    print ("Got %s percent correct!" % str(correct))
    metascore.append(correct)

print "\nEnd score:", np.mean(metascore), "percent correct!"

Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 22 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 16 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 17 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 16 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 27 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 29 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 17 percent correct!
Training Eigen face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 25 p

In [9]:
LBPHFaces = cv2.createLBPHFaceRecognizer()

def runLBPHFaces():
    trainingSetFiles, trainingEmotions, testingSetFiles, testingEmotions = createSets()
    
    print "Training LBPH face classifier..."
    print "Size of training set is:", len(trainingEmotions), "images"
    LBPHFaces.train(trainingSetFiles, np.asarray(trainingEmotions))

    print "Predicting classification set..."
    count = 0
    correct = 0
    incorrect = 0
    for image in testingSetFiles:
        pred, conf = LBPHFaces.predict(image)
        if pred == testingEmotions[count]:
            correct += 1
            count += 1
        else:
            incorrect += 1
            count += 1
    return ((100*correct)/(correct + incorrect))

#Now run it
metascore = []
for i in range(0,5):
    correct = runLBPHFaces()
    print ("Got %s percent correct!" % str(correct))
    metascore.append(correct)

print "\nEnd score:", np.mean(metascore), "percent correct!"

Training LBPH face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 22 percent correct!
Training LBPH face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 30 percent correct!
Training LBPH face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 24 percent correct!
Training LBPH face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 20 percent correct!
Training LBPH face classifier...
Size of training set is: 260 images
Predicting classification set...
Got 19 percent correct!

End score: 23.0 percent correct!
